# GRUANpy Proof of Concept

GRUANpy is a python toolkit to facilitate the analysis of GRUAN data. It includes several functionalities and is easily extensible thanks to its structure based on the inclusion of different data models and helper classes that provide specialized methods for different types of purposes. The different functions can also be executed in succession in order to create real data pipelines that allow a large variety of outputs.

Here is a list of the main features identified so far:
- download, the ability to download data of interest
- merge, the ability to merge data from different products
- aggregation, the ability to aggregate different observations and lower the resolution of the data (respecting the GRUAN principles in uncertainty processing)

In [1]:
# Demo setup
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from gruanpy import GRUANpy
from datetime import datetime
download_folder=r"gdp_demo_090625"
gp = GRUANpy(download_folder=download_folder)
%matplotlib qt


## DOWNLOAD

In [ ]:
# Download Gruan Data Product (GDP) through NOAA FTP
ftp_dir_path=r'pub/data/gruan/processing/level2/RS92-GDP/version-002/LIN/2018'
files=gp.search(ftp_dir_path=ftp_dir_path)
print(f"Found {len(files)} files in {ftp_dir_path}")
for file in files[:5]:
    print(file)
print("-----"*10)
print("Downloading first 2 files for demo purpose")
files=files[:2]
for file in files:
    gp.download(ftp_dir_path=ftp_dir_path, file_name=file)
print("Download completed.")
print("-----"*10)
for file in files:
    gdp=gp.read(download_folder+r'/ '[:-1]+file)
    print(gdp.global_attrs.head())
    print("-----"*10)

# Iterative script at code_examples\download_gdp.py

Found 75 files in pub/data/gruan/processing/level2/RS92-GDP/version-002/LIN/2018
LIN-RS-01_2_RS92-GDP_002_20180611T180000_1-002-001.nc
LIN-RS-01_2_RS92-GDP_002_20180103T000000_1-002-002.nc
LIN-RS-01_2_RS92-GDP_002_20180612T002400_1-000-002.nc
LIN-RS-01_2_RS92-GDP_002_20180122T120000_1-002-001.nc
LIN-RS-01_2_RS92-GDP_002_20180613T180000_1-002-002.nc
--------------------------------------------------
Download completed.
--------------------------------------------------
     Attribute                                              Value
0  Conventions                                             CF-1.4
1        title                RS92 GRUAN Data Product (Version 2)
2  institution  MOL - Lindenberg Meteorological Observatory; D...
3       source                                           RS92-SGP
4      history  2018-06-11 21:30:51.000Z RS92-GDP: RS92 GRUAN ...
--------------------------------------------------
     Attribute                                              Value
0  Conventions

In [ ]:
# Download GRUAN DATA through CDS API in csv format

api_response_file = "api_response.csv"
api_request = """
import cdsapi

dataset = "insitu-observations-gruan-reference-network"
request = {
    "variable": [
        "air_temperature",
        "relative_humidity",
        "relative_humidity_effective_vertical_resolution",
        "wind_speed",
        "wind_from_direction",
        "eastward_wind_speed",
        "northward_wind_speed",
        "shortwave_radiation",
        "air_pressure",
        "altitude",
        "geopotential_height",
        "frost_point_temperature",
        "water_vapour_volume_mixing_ratio",
        "vertical_speed_of_radiosonde",
        "time_since_launch"
    ],
    "year": "2014",
    "month": "10",
    "day": ["14"],
    "data_format": "csv",
    "area": [90, 0, 0, 90]
}

client = cdsapi.Client()
"""+f"""
target= r"{download_folder}\\{api_response_file}" # Change this to your desired output path
client.retrieve(dataset, request, target)

"""
print("Executing API request ...")
gp.exec_request(api_request)
print("-----"*10)


Executing API request ...


2025-05-28 10:53:45,454 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-05-28 10:53:45,455 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.
2025-05-28 10:53:46,171 INFO Request ID is 1cee6683-13e9-4835-83c8-6c0deddbaf89
2025-05-28 10:53:46,275 INFO status has been updated to accepted
2025-05-28 10:55:02,255 INFO status has been updated to successful
                                                                                          

--------------------------------------------------


In [6]:
# Download GRUAN DATA through CDS API in netCDF format

api_response_file = "api_response.nc"
api_request = """
import cdsapi

dataset = "insitu-observations-gruan-reference-network"
request = {
    "variable": [
        "air_temperature",
        "relative_humidity",
        "relative_humidity_effective_vertical_resolution",
        "wind_speed",
        "wind_from_direction",
        "eastward_wind_speed",
        "northward_wind_speed",
        "shortwave_radiation",
        "air_pressure",
        "altitude",
        "geopotential_height",
        "frost_point_temperature",
        "water_vapour_volume_mixing_ratio",
        "vertical_speed_of_radiosonde",
        "time_since_launch"
    ],
    "year": "2014",
    "month": "10",
    "day": ["14"],
    "data_format": "netcdf",
    "area": [90, 0, 0, 90]
}

client = cdsapi.Client()
"""+f"""
target= r"{download_folder}\\{api_response_file}" # Change this to your desired output path
client.retrieve(dataset, request, target)

"""
print("Executing API request ...")
gp.exec_request(api_request)
print("-----"*10)

Executing API request ...


2025-05-28 11:16:44,989 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-05-28 11:16:44,990 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.
2025-05-28 11:16:45,706 INFO Request ID is f5bfd683-46f5-40fb-bca0-50816aa87b21
2025-05-28 11:16:45,809 INFO status has been updated to accepted
2025-05-28 11:17:35,883 INFO status has been updated to successful
                                                                                         

--------------------------------------------------


# MERGE and AGGREGATE

In [2]:
# GDP RS41 spatial gridding
file_path = r'gdp_demo_090625\LIN-RS-01_2_RS41-GDP_001_20141209T120000_1-009-002.nc'
gdp=gp.read(file_path)

bin_column = 'press' # Choose the binning column (alt or press)
target_columns = ['temp', 'rh']

ggd = gp.spatial_gridding(gdp, bin_column, target_columns)

# Plot original and gridded data
for column in target_columns:
    fig, ax1 = plt.subplots(figsize=(7, 6))
    if bin_column == 'press':
        ax1.set_yscale('log')
        ax1.invert_yaxis()
    ax1.scatter(gdp.data[column], gdp.data[bin_column], label='Original Data', alpha=0.5)
    ax1.scatter(ggd.data[column], ggd.data[bin_column], label='Gridded Data', color='red', alpha=0.5)
    ax1.fill_betweenx(gdp.data[bin_column], gdp.data[column] - gdp.data[column+'_uc'], gdp.data[column] + gdp.data[column+'_uc'], color='blue', alpha=0.2, label='Original Uncertainty')
    ax1.fill_betweenx(ggd.data[bin_column], ggd.data[column] - ggd.data[column+'_uc'], ggd.data[column] + ggd.data[column+'_uc'], color='red', alpha=0.2, label='Gridded Uncertainty')
    ax1.set_xlabel(f'{column.capitalize()} {gdp.variables_attrs[gdp.variables_attrs['variable'] == column]['units'].values[0]}')
    ax1.set_ylabel(f'{bin_column.capitalize()} {gdp.variables_attrs[gdp.variables_attrs['variable'] == bin_column]['units'].values[0]}')
    ax1.legend()
    long_name= gdp.variables_attrs[gdp.variables_attrs['variable'] == column]['long_name'].values[0]
    ax1.set_title(f'{long_name} Mandatory Levels Spatial Gridding')

    plt.tight_layout()
    plt.show()

In [3]:
# GDP RS41 temporal gridding

# Read multiple GDP files
gdp_folder=r'C:\Users\tomma\Documents\SDC\Repos\GRUAN_EDA\gdp\products_RS41-GDP-1_LIN_2017'#gdp_demo_090625\products_RS41-GDP-1_LIN_2017' # Path to the folder
gdp_files = [os.path.join(gdp_folder, f) for f in os.listdir(gdp_folder) if f.endswith('.nc')]
gdps=[]
for file in tqdm(gdp_files[:50] , desc="Reading GDPs"):
    gdps.append(gp.read(file))

Reading GDPs: 100%|██████████| 50/50 [01:08<00:00,  1.37s/it]


In [4]:
# Uniform Spatial gridding accross multiple GDPs
ggds=[]
target_columns = ['temp', 'rh']
for gdp in tqdm(gdps, desc="Spatial Gridding"):
    ggd = gp.spatial_gridding(gdp, 'press', target_columns, mandatory_levels_flag=True)
    ggds.append(ggd)

Spatial Gridding: 100%|██████████| 50/50 [00:01<00:00, 28.68it/s]


In [25]:
# Merge all gridded data
mggd=pd.DataFrame()
for ggd in tqdm(ggds, desc="Merging Gridded Data"):
    start_time_str = ggd.metadata[ggd.metadata['Attribute'] == 'g.Measurement.StartTime']['Value'].values[0]
    start_time = datetime.strptime(start_time_str, "%Y-%m-%dT%H:%M:%S.%fZ")
    ggd_data = ggd.data.copy()
    ggd_data['time'] = start_time
    mggd = pd.concat([mggd, ggd_data], ignore_index=True)

# Plotting the merged gridded data
for column in target_columns:
    # Separate data into day and night based on time (e.g., 6:00-18:00 as day, else night)
    mand_lvl_val = 1000
    subset = mggd[mggd['mand_lvl'] == mand_lvl_val].copy()
    subset['hour'] = subset['time'].dt.hour
    day_data = subset[(subset['hour'] >= 6) & (subset['hour'] < 18)]
    night_data = subset[(subset['hour'] < 6) | (subset['hour'] >= 18)]

    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(day_data['time'], day_data[column], marker='o', linestyle='-', label=f'{column} (Day)')
    ax.plot(night_data['time'], night_data[column], marker='x', linestyle='--', label=f'{column} (Night)')
    ax.fill_between(day_data['time'], day_data[column] - day_data[column + '_uc'], day_data[column] + day_data[column + '_uc'], alpha=0.2, label='Day Uncertainty')
    ax.fill_between(night_data['time'], night_data[column] - night_data[column + '_uc'], night_data[column] + night_data[column + '_uc'], alpha=0.2, label='Night Uncertainty')
    ax.set_xlabel('Time')
    ax.set_ylabel(column.capitalize())
    ax.set_title(f'{column.capitalize()} at Mandatory Level {mand_lvl_val} Over Time (Day vs Night)')
    ax.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

Merging Gridded Data: 100%|██████████| 50/50 [00:00<00:00, 899.27it/s]


In [26]:
mggd.head()

,mand_lvl,temp,rh,press,temp_uc_ucor_avg,temp_var,temp_uc_ucor,temp_uc_scor,temp_uc_tcor,temp_uc,rh_uc_ucor_avg,rh_var,rh_uc_ucor,rh_uc_scor,rh_uc_tcor,rh_uc,time
0,5,216.646744,1.314128,5.972275,0.003949,0.137854,0.137911,0.0,0.079224,0.159047,0.049452,0.017888,0.052588,0,2.098747,2.099405,2017-01-02 22:45:18.235
1,7,212.783279,1.692838,7.040342,0.001748,0.071990,0.072011,0.0,0.080095,0.107707,0.013101,0.009679,0.016289,0,2.191129,2.191190,2017-01-02 22:45:18.235
2,10,201.235672,3.311255,11.141134,0.001247,0.125640,0.125646,0.0,0.083302,0.150752,0.020167,0.032528,0.038273,0,2.621775,2.622054,2017-01-02 22:45:18.235
3,20,199.591782,5.000197,19.690186,0.001235,0.147285,0.147290,0.0,0.083829,0.169475,0.028827,0.071766,0.077339,0,2.801397,2.802464,2017-01-02 22:45:18.235
4,30,199.036987,5.941062,31.834671,0.000644,0.081231,0.081233,0.0,0.083982,0.116841,0.028855,0.042116,0.051053,0,2.803793,2.804258,2017-01-02 22:45:18.235


In [31]:
# Temporal gridding from scratch

bin_size = 7 # Size of the time bin in days
mggd[bin] = (mggd['time'].dt.day // bin_size) * bin_size + bin_size / 2
first_data = mggd['time'].min()
tggd = mggd.groupby(['mand_lvl', 'time_bin'])[target_columns].mean().reset_index() # 3.12
tggd['time'] = pd.to_datetime(tggd['time_bin'], unit='D', origin=first_data)
for col in target_columns:
    tggd[col + '_uc_ucor_avg'] = mggd.groupby([bin,'mand_lvl'])[col + '_uc_ucor'].apply(
                lambda x: (((x**2).sum())**0.5)/len(x)
                ).reset_index(drop=True) #3.13
    tggd[col + '_var'] = mggd.groupby([bin,'mand_lvl'])[col].apply(
                lambda x: ((((x-x.mean())**2).sum())/(len(x)*max((len(x)-1),1)))**0.5
                ).reset_index(drop=True) #3.14
    tggd[col + '_uc_sc']=mggd.groupby([bin,'mand_lvl'])[col + '_uc_scor'].apply(
                lambda x: (((x**2).sum())**0.5)/len(x)
                ).reset_index(drop=True) #3.15
    tggd[col + '_uc_ucor']=(
        tggd[col+'_uc_ucor_avg']**2 + tggd[col + '_var']**2 + tggd[col + '_uc_sc']**2)**0.5 #3.16
    tggd[col + '_cor']=mggd.groupby([bin,'mand_lvl'])[col + '_uc_tcor'].mean().reset_index(drop=True) #3.17
    tggd[col+'_uc']=(
        tggd[col+'_uc_ucor']**2 + tggd[col+'_cor']**2)**0.5 #3.18

In [32]:
tggd.head()

,mand_lvl,time_bin,temp,rh,time,temp_uc_ucor_avg,temp_var,temp_uc_sc,temp_uc_ucor,temp_cor,temp_uc,rh_uc_ucor_avg,rh_var,rh_uc_sc,rh_uc_ucor,rh_cor,rh_uc
0,3,10.5,236.506180,0.199481,2017-01-12 12:00:00,0.113402,4.317035,0.067530,4.319052,0.246014,4.326053,0.011174,0.208364,0.0,0.208664,1.510194,1.524542
1,3,24.5,233.541946,0.493441,2017-01-26 12:00:00,0.033854,2.295907,0.046920,2.296635,0.145872,2.301263,0.008353,0.187165,0.0,0.187351,1.387363,1.399956
2,5,3.5,232.249100,0.499769,2017-01-05 12:00:00,0.042164,2.873644,0.042834,2.874272,0.125313,2.877003,0.007464,0.329356,0.0,0.329440,1.601413,1.634948
3,5,10.5,224.368484,0.900499,2017-01-12 12:00:00,0.052329,1.496945,0.036161,1.498296,0.110590,1.502372,0.016069,0.441035,0.0,0.441328,1.913264,1.963505
4,5,17.5,233.200150,0.622524,2017-01-19 12:00:00,0.023436,0.962533,0.032480,0.963366,0.109934,0.969618,0.013163,0.455913,0.0,0.456103,2.020304,2.071149


In [34]:
print(mggd.shape,tggd.shape)

(851, 18) (89, 17)


## PIPELINE

# DATA VISUALIZATION